In [30]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam

##### About Data
Wisconsin Breast Cancer dataset has 699 observations collected using fine-needle tissue from a mass under skin. It has 11 variables of which nine are predictor variables (cytological characteristics used to identify mass as benign or malign), ID and a class variable (has values 2 for benign, 4 for malignant). 458 of the samples are benign and 241 are malignant. <br/>

There are 16 samples with missing data. Data file doesn't have column names and they are listed in a separate file. <br/>
'class' variable is the outcome (2=benign, 4=malignant).

In [31]:
data = pd.read_csv('data/breast-cancer-wisconsin.data.txt', sep=",", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


###### Adding column headers to data

In [32]:
data.columns = ["ID", "clumpThickness", "sizeUniformity",
"shapeUniformity", "maginalAdhesion",
"singleEpithelialCellSize", "bareNuclei",
"blandChromatin", "normalNucleoli", "mitosis", "class"]
data.head(10)

,ID,clumpThickness,sizeUniformity,shapeUniformity,maginalAdhesion,singleEpithelialCellSize,bareNuclei,blandChromatin,normalNucleoli,mitosis,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [33]:
data.shape

(699, 11)

###### Counting the different classes of 'class' variable

In [34]:
data['class'].value_counts()

2    458
4    241
Name: class, dtype: int64

###### Converting classes '2','4' to binary '0'(benign) and '1'(malignant)

In [35]:
data.dtypes

ID                           int64
clumpThickness               int64
sizeUniformity               int64
shapeUniformity              int64
maginalAdhesion              int64
singleEpithelialCellSize     int64
bareNuclei                  object
blandChromatin               int64
normalNucleoli               int64
mitosis                      int64
class                        int64
dtype: object

######  Converting 'class' variable from int to string

In [36]:
data['class'] = data['class'].astype(str)

In [37]:
#Converting 'object' to numeric data type
data['bareNuclei'] = pd.to_numeric(data['bareNuclei'], errors='coerce')
#data['bareNuclei'] = data['bareNuclei'].astype(str).astype(int)

In [38]:
data.dtypes

ID                            int64
clumpThickness                int64
sizeUniformity                int64
shapeUniformity               int64
maginalAdhesion               int64
singleEpithelialCellSize      int64
bareNuclei                  float64
blandChromatin                int64
normalNucleoli                int64
mitosis                       int64
class                        object
dtype: object

In [39]:
data.isnull().sum()

ID                           0
clumpThickness               0
sizeUniformity               0
shapeUniformity              0
maginalAdhesion              0
singleEpithelialCellSize     0
bareNuclei                  16
blandChromatin               0
normalNucleoli               0
mitosis                      0
class                        0
dtype: int64

###### Replacing missing values with mean value of column

In [40]:
data["bareNuclei"].fillna(data["bareNuclei"].mean(), inplace=True)

In [41]:
#Now check for missing values in dataframe
data.isnull().sum()

ID                          0
clumpThickness              0
sizeUniformity              0
shapeUniformity             0
maginalAdhesion             0
singleEpithelialCellSize    0
bareNuclei                  0
blandChromatin              0
normalNucleoli              0
mitosis                     0
class                       0
dtype: int64

In [42]:
data = data.drop(['ID'], axis=1)
data.columns

Index(['clumpThickness', 'sizeUniformity', 'shapeUniformity',
       'maginalAdhesion', 'singleEpithelialCellSize', 'bareNuclei',
       'blandChromatin', 'normalNucleoli', 'mitosis', 'class'],
      dtype='object')

In [43]:
data.describe()

,clumpThickness,sizeUniformity,shapeUniformity,maginalAdhesion,singleEpithelialCellSize,bareNuclei,blandChromatin,normalNucleoli,mitosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.601852,2.438364,3.053634,1.715078
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [44]:
data['mitosis'].value_counts()

1     579
2      35
3      33
10     14
4      12
7       9
8       8
5       6
6       3
Name: mitosis, dtype: int64

In [45]:
#np.ravel(data['class'])

###### Splitting data into input and output variables

In [46]:
X = data.iloc[:,0:9].values
y = data.iloc[:,9].values

In [47]:
#encoding class values as integers
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values

###### Splitting data into train and test variables

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=0) 

In [49]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(489, 9)
(489, 2)
(210, 9)
(210, 2)


In [50]:
model = Sequential()

model.add(Dense(10, input_shape=(9,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [51]:
model.compile(optimizer=Adam(lr=0.04), loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                100       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 18        
Total params: 206
Trainable params: 206
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
489/489 [==============================] - 0s 41us/step - loss: 0.0512 - acc: 0.9775
Epoch 2/200
489/489 [==============================] - 0s 43us/step - loss: 0.0499 - acc: 0.9816
Epoch 3/200
489/489 [==============================] - 0s 43us/step - loss: 0.0714 - acc: 0.9714
Epoch 4/200
489/489 [==============================] - 0s 41us/step - loss: 0.0504 - acc: 0.9836
Epoch 5/200
489/489 [==============================] - 0s 45us/step - loss: 0.0460 - acc: 0.9796
Epoch 6/200
489/489 [==============================] - 0s 39us/step - loss: 0.0461 - acc: 0.9796
Epoch 7/200
489/489 [==============================] - 0s 43us/step - loss: 0.0458 - acc: 0.9836
Epoch 8/200
489/489 [==============================] - 0s 45us/step - loss: 0.0566 - acc: 0.9755
Epoch 9/200
489/489 [==============================] - ETA: 0s - loss: 0.0336 - acc: 0.968 - 0s 41us/step - loss: 0.0565 - acc: 0.9775
Epoch 10/200
489/489 [==============================] - 0s 45us/step - loss: 0.0510 - acc

489/489 [==============================] - 0s 39us/step - loss: 0.0585 - acc: 0.9796
Epoch 168/200
489/489 [==============================] - 0s 39us/step - loss: 0.0570 - acc: 0.9836
Epoch 169/200
489/489 [==============================] - 0s 39us/step - loss: 0.0555 - acc: 0.9693
Epoch 170/200
489/489 [==============================] - 0s 47us/step - loss: 0.0568 - acc: 0.9775
Epoch 171/200
489/489 [==============================] - 0s 43us/step - loss: 0.0477 - acc: 0.9714
Epoch 172/200
489/489 [==============================] - 0s 41us/step - loss: 0.0510 - acc: 0.9816
Epoch 173/200
489/489 [==============================] - 0s 43us/step - loss: 0.0511 - acc: 0.9775
Epoch 174/200
489/489 [==============================] - 0s 41us/step - loss: 0.0442 - acc: 0.9816
Epoch 175/200
489/489 [==============================] - 0s 43us/step - loss: 0.0477 - acc: 0.9816
Epoch 176/200
489/489 [==============================] - 0s 43us/step - loss: 0.0475 - acc: 0.9816
Epoch 177/200
489/489 [=

In [55]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

210/210 [==============================] - 0s 310us/step

acc: 95.24%


In [56]:
y_pred = model.predict(X_test)

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)

In [57]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_class, y_pred_class))
print(confusion_matrix(y_test_class, y_pred_class))

             precision    recall  f1-score   support

          0       0.98      0.95      0.96       135
          1       0.91      0.96      0.94        75

avg / total       0.95      0.95      0.95       210

[[128   7]
 [  3  72]]
